In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
import gc
from numba import njit
import tqdm
import copy
import datetime
%matplotlib inline


In [2]:
ss = pd.read_csv('sample_submission.csv')

In [3]:
city_df = pd.read_csv('tab_6_city.csv')

In [4]:
users_df = pd.read_csv('./kaggle_tab_1345/tab_4_user_profiles.csv', parse_dates=['bdate'])

In [5]:
categories_df = pd.read_csv('./kaggle_tab_1345/tab_3_categories.csv')

In [6]:
product_props_df = pd.read_csv('./kaggle_tab_1345/tab_5_product_properties.csv')

In [7]:
orders_df = pd.read_csv('./kaggle_tab_1345/tab_1_orders.csv', parse_dates=['order_created_time'])
orders_df = orders_df[orders_df.user_id.isin(ss.Id)]
orders_df = orders_df[orders_df.order_id.map(orders_df.order_id.value_counts() <= 1)]

In [8]:
products_tables = []

for product_table_name in sorted(glob.glob('./sbermarket*/*.csv')):
    products_tables.append(pd.read_csv(product_table_name))
    

product_df = pd.concat(products_tables)

del products_tables
gc.collect()

product_df = product_df[product_df.user_id.isin(ss.Id)]


0

In [11]:
product_df = product_df.merge(orders_df, how='left', on='order_id')

product_df = product_df.rename(columns={'user_id_x':'user_id'})
product_df.drop(labels='user_id_y', axis=1, inplace=True)


In [12]:

product_df = product_df.merge(users_df, how='left', on='user_id')
product_df.retailer = product_df.retailer.fillna('Unknown')

product_df.retailer[
    (product_df.retailer != 'Лента') & (product_df.retailer != 'METRO') & (product_df.retailer != 'Ашан') &  (product_df.retailer != 'Unknown') 
] = 'SMALL'

product_df_2 = product_df.set_index(['order_id', 'product_id'])
product_df = product_df_2[~product_df_2.index.duplicated()].reset_index()

del product_df_2
gc.collect()
#product_df = product_df[product_df.product_id.map(product_df.product_id.value_counts()) > 30]  #? заменить на макса


<ipython-input-12-67018a050a2e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df.retailer[


0

In [13]:
# prod_statistic = product_df.groupby(['user_id','product_id']).order_created_time.nunique()
# key_prod_id = prod_statistic[prod_statistic.values >= 3].reset_index().product_id.unique()
# product_df = product_df[product_df.product_id.isin(key_prod_id)]

In [14]:
product_df.store_id.fillna(-1, inplace=True)
main_retailer_for_user = product_df.groupby('user_id').store_id.agg(lambda x: x.value_counts().index[0])


top_products = product_df.groupby('store_id').product_id.value_counts()
retail_list = list(product_df.store_id.unique())

top_products_by_retail = {retail: list(top_products[retail][:50].index.values) for retail in retail_list}

In [15]:
top_products = product_df.product_id.value_counts()
tp = list(top_products[:50].index)

In [16]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

number_of_orders_per_user = product_df.groupby('user_id').order_id.nunique()
product_df['ones'] = 1/number_of_orders_per_user[product_df.user_id].values

#product_df['ones'] = 1


user = CategoricalDtype(sorted(product_df.user_id.unique()), ordered=True)
product = CategoricalDtype(sorted(product_df.product_id.unique()), ordered=True)

row = product_df.user_id.astype(user).cat.codes
col = product_df.product_id.astype(product).cat.codes
sparse_matrix = csr_matrix((product_df["ones"], (row, col)), \
                           shape=(user.categories.size, product.categories.size))

user_to_idx = pd.Series(index=user.categories, data=np.arange(len(user.categories)))

In [17]:
import implicit

# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=256)

# train the model on a sparse matrix of item/user/confidence weights
model.fit(sparse_matrix.T.tocsr())

# recommend items for a user
#user_items = sparse_matrix.T.tocsr()
#recommendations = model.recommend(0, user_items)

# find related items
#related = model.similar_items(itemid)

In [18]:
user_product_series = product_df.groupby('user_id').product_id.value_counts()
user_product_series = user_product_series.groupby('user_id').head(50)
user_product_lists = user_product_series.index.values

In [20]:
def get_top_products(lists):
    us_prod_dict = {}

    for i in tqdm.tqdm(range(len(lists))):
        u, p = lists[i]
        try:
            us_prod_dict[u].append(p)
        except:
            us_prod_dict[u] = [p]
    
    for u in tqdm.tqdm(us_prod_dict):
        len_us = len(us_prod_dict[u])
        if len_us < 50:
            sub_tp = [x for x in top_products_by_retail[main_retailer_for_user[u]] if x not in us_prod_dict[u]]
            us_prod_dict[u] = us_prod_dict[u] + sub_tp[:50-len_us]
            
            assert len(us_prod_dict[u]) == len(set(us_prod_dict[u])), u
            
    for u in us_prod_dict:
        us_prod_dict[u] = ' '.join(map(str, us_prod_dict[u]))
    return us_prod_dict

In [21]:
def get_top_products_svd(lists):
    us_prod_dict = {}

    for i in tqdm.tqdm(range(len(lists))):
        u, p = lists[i]
        try:
            us_prod_dict[u].append(p)
        except:
            us_prod_dict[u] = [p]
    
    for u in tqdm.tqdm(us_prod_dict):
        len_us = len(us_prod_dict[u])
        if len_us < 50:
            recommends = list(product.categories[[it[0] for it in model.recommend(user_to_idx[u], sparse_matrix, N=50)]].values)
            sub_tp = [x for x in recommends if x not in us_prod_dict[u]]
            us_prod_dict[u] = us_prod_dict[u] + sub_tp[:50-len_us]
        
    for u in us_prod_dict:
        us_prod_dict[u] = ' '.join(map(str, us_prod_dict[u]))
    return us_prod_dict

In [22]:
us_prod_dict = get_top_products_svd(user_product_lists)

100%|██████████| 107068/107068 [02:23<00:00, 744.26it/s]


In [23]:
sub = pd.DataFrame(index = us_prod_dict.keys(), data=us_prod_dict.values())
sub = sub.reset_index()
sub.columns = ss.columns
sub.to_csv('baseline_submission_2.csv', index=False)

!kaggle competitions submit -c test-recsys -f baseline_submission_2.csv -m "Baseline model. ALS. Big table. Big model"

100%|██████████████████████████████████████| 32.2M/32.2M [00:07<00:00, 4.44MB/s]
Successfully submitted to Predict products in the next order 